# Data Preprocessing

In [23]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
from sklearn.preprocessing import StandardScaler, MinMaxScaler

### Player Valuations dataset

In [24]:
player_valuations = pd.read_csv('../data/preprocessed_valuations.csv')
player_valuations.drop(columns='Unnamed: 0', inplace=True)
player_valuations.head()

,name,last_season,average_value
0,A.J. Soares,2016.0,2.187500e+05
1,AJ Leitch-Smith,2017.0,1.214286e+05
2,Aaron Appindangoyé,2023.0,8.333333e+05
3,Aaron Bastiaans,2020.0,1.656250e+05
4,Aaron Boupendza,2020.0,4.266667e+06


In [25]:
# Global historical inflation rate. Min year is 2012
inflation_rates = {
    2012: 0.0373,
    2013: 0.0262,
    2014: 0.0235,
    2015: 0.0143,
    2016: 0.0155,
    2017: 0.0219,
    2018: 0.0244,
    2019: 0.0221,
    2020: 0.0193,
    2021: 0.0348,
    2022: 0.0827,
}

def adjust_for_inflation(value, year):
    adjusted_value = value
    for y in range(int(year) + 1, max(inflation_rates.keys()) + 1):
        adjusted_value *= (1 + inflation_rates[y])
    return adjusted_value

player_valuations_adjusted = player_valuations.copy()

if 'adjusted_value' not in player_valuations_adjusted.columns:

    player_valuations_adjusted['adjusted_value'] = player_valuations_adjusted.apply(lambda row: adjust_for_inflation(row['average_value'], row['last_season']), axis=1)
    player_valuations_adjusted.drop(columns = 'average_value', inplace=True)
    


In [26]:
player_valuations

,name,last_season,average_value
0,A.J. Soares,2016.0,2.187500e+05
1,AJ Leitch-Smith,2017.0,1.214286e+05
2,Aaron Appindangoyé,2023.0,8.333333e+05
3,Aaron Bastiaans,2020.0,1.656250e+05
4,Aaron Boupendza,2020.0,4.266667e+06
...,...,...,...
28686,Ümit Kurt,2016.0,8.540000e+05
28687,Ümit Türker,2016.0,5.000000e+04
28688,Ümran Zambak,2019.0,5.000000e+04
28689,Üstün Bilgi,2013.0,1.602941e+05


In [27]:
player_valuations_adjusted

,name,last_season,adjusted_value
0,A.J. Soares,2016.0,2.672920e+05
1,AJ Leitch-Smith,2017.0,1.451946e+05
2,Aaron Appindangoyé,2023.0,8.333333e+05
3,Aaron Bastiaans,2020.0,1.855626e+05
4,Aaron Boupendza,2020.0,4.780279e+06
...,...,...,...
28686,Ümit Kurt,2016.0,1.043508e+06
28687,Ümit Türker,2016.0,6.109532e+04
28688,Ümran Zambak,2019.0,5.710006e+04
28689,Üstün Bilgi,2013.0,2.064856e+05


In [28]:
num_columns = ['last_season', 'adjusted_value']
num_data = player_valuations_adjusted[num_columns]
scaler = StandardScaler().fit(num_data)
scaled_valuations = scaler.transform(num_data)

# Replace original data with scaled data 
player_valuations_adjusted_scaled = player_valuations_adjusted.copy()
player_valuations_adjusted_scaled[num_columns] = scaled_valuations   
player_valuations_adjusted_scaled

,name,last_season,adjusted_value
0,A.J. Soares,-0.752289,-0.333430
1,AJ Leitch-Smith,-0.479310,-0.362391
2,Aaron Appindangoyé,1.158566,-0.199166
3,Aaron Bastiaans,0.339628,-0.352816
4,Aaron Boupendza,0.339628,0.737044
...,...,...,...
28686,Ümit Kurt,-0.752289,-0.149313
28687,Ümit Türker,-0.752289,-0.382339
28688,Ümran Zambak,0.066649,-0.383287
28689,Üstün Bilgi,-1.571227,-0.347853


### Player skill dataset

In [29]:
player_ratings = pd.read_csv('../data/FUT_player_data.csv')
pd.set_option('display.max_columns', None)
player_ratings.head()

,id,futbin_id,name,height,weight,age,club,league,nation,rarity,position,foot,attackWorkRate,defenseWorkRate,cardColor,overallRating,pace,shooting,passing,dribbling,defending,physicality,pace_acceleration,pace_sprintSpeed,shooting_positioning,shooting_finishing,shooting_shotPower,shooting_longShots,shooting_volleys,shooting_penalties,passing_vision,passing_crossing,passing_freeKickAccuracy,passing_shortPassing,passing_longPassing,passing_curve,dribbling_agility,dribbling_balance,dribbling_reactions,dribbling_ballControl,dribbling_dribbling,dribbling_composure,defending_interceptions,defending_headingAccuracy,defending_standingTackle,defending_slidingTackle,defending_defenseAwareness,phsyicality_jumping,physicality_stamina,physicality_strength,physicality_aggression,goalkeeper_diving,goalkeeper_handling,goalkeeper_kicking,goalkeeper_positioning,goalkeeper_reflexes,goalkeeper_speed
0,18949,54231,Kylian Mbappé,182,73,24,73,16,18,16,ST,Right,High,Low,gold,99,99,98,92,99,45,87,99.0,99.0,99.0,99.0,99.0,94.0,95.0,93.0,95.0,90.0,80.0,97.0,82.0,92.0,99.0,91.0,99.0,99.0,99.0,99.0,48.0,87.0,43.0,40.0,33.0,88.0,99.0,87.0,73.0,NaN,NaN,NaN,NaN,NaN,99
1,18981,54251,Karim Benzema,185,81,35,607,350,18,164,CF,Right,High,Med,gold,99,97,99,93,98,50,97,97.0,97.0,99.0,99.0,99.0,97.0,99.0,97.0,99.0,84.0,83.0,98.0,86.0,93.0,91.0,90.0,99.0,99.0,99.0,99.0,55.0,99.0,33.0,25.0,56.0,99.0,99.0,99.0,90.0,NaN,NaN,NaN,NaN,NaN,97
2,18982,54249,Zinedine Zidane,185,77,51,112658,2118,18,171,CAM,Right,Med,Med,gold,99,92,96,99,97,87,90,93.0,92.0,96.0,95.0,95.0,99.0,97.0,94.0,99.0,99.0,99.0,99.0,99.0,99.0,88.0,90.0,99.0,99.0,99.0,99.0,95.0,99.0,88.0,72.0,83.0,87.0,94.0,92.0,83.0,NaN,NaN,NaN,NaN,NaN,92
3,18730,54005,Pelé,173,70,82,112658,2118,54,153,LW,Right,High,Med,gold,99,96,97,94,99,61,78,96.0,96.0,98.0,99.0,95.0,95.0,96.0,94.0,98.0,91.0,90.0,97.0,89.0,90.0,97.0,96.0,99.0,99.0,99.0,99.0,68.0,96.0,54.0,50.0,56.0,90.0,91.0,78.0,61.0,NaN,NaN,NaN,NaN,NaN,96
4,19001,54277,Robert Lewandowski,185,81,35,241,53,37,164,ST,Right,High,Med,gold,99,97,99,92,97,53,99,98.0,97.0,99.0,99.0,99.0,99.0,99.0,99.0,94.0,83.0,99.0,98.0,82.0,92.0,87.0,92.0,99.0,99.0,97.0,99.0,60.0,99.0,52.0,24.0,42.0,99.0,97.0,99.0,99.0,NaN,NaN,NaN,NaN,NaN,97


In [30]:
#substitute the foreign keys with the actual values 

#read csv files as dicts
club_ids = pd.read_csv('../data/club_ids.csv', index_col = 'id')
league_ids = pd.read_csv('../data/league_ids.csv', index_col = 'id')
nation_ids = pd.read_csv('../data/nation_ids.csv', index_col = 'id')

club_ids =  {k:v[0] for k, v in zip(club_ids.index, club_ids.values)}
league_ids = {k:v[0] for k, v in zip(league_ids.index, league_ids.values)}
nation_ids = {k:v[0] for k, v in zip(nation_ids.index, nation_ids.values)}

# #this is a master dictionary 
master_dict = {'club': club_ids, 'league': league_ids, 'nation': nation_ids}

# #replace the values 
player_ratings.replace(master_dict)

,id,futbin_id,name,height,weight,age,club,league,nation,rarity,position,foot,attackWorkRate,defenseWorkRate,cardColor,overallRating,pace,shooting,passing,dribbling,defending,physicality,pace_acceleration,pace_sprintSpeed,shooting_positioning,shooting_finishing,shooting_shotPower,shooting_longShots,shooting_volleys,shooting_penalties,passing_vision,passing_crossing,passing_freeKickAccuracy,passing_shortPassing,passing_longPassing,passing_curve,dribbling_agility,dribbling_balance,dribbling_reactions,dribbling_ballControl,dribbling_dribbling,dribbling_composure,defending_interceptions,defending_headingAccuracy,defending_standingTackle,defending_slidingTackle,defending_defenseAwareness,phsyicality_jumping,physicality_stamina,physicality_strength,physicality_aggression,goalkeeper_diving,goalkeeper_handling,goalkeeper_kicking,goalkeeper_positioning,goalkeeper_reflexes,goalkeeper_speed
0,18949,54231,Kylian Mbappé,182,73,24,Paris SG,Ligue 1,France,16,ST,Right,High,Low,gold,99,99,98,92,99,45,87,99.0,99.0,99.0,99.0,99.0,94.0,95.0,93.0,95.0,90.0,80.0,97.0,82.0,92.0,99.0,91.0,99.0,99.0,99.0,99.0,48.0,87.0,43.0,40.0,33.0,88.0,99.0,87.0,73.0,NaN,NaN,NaN,NaN,NaN,99
1,18981,54251,Karim Benzema,185,81,35,Al Ittihad,MBS Pro League (SAU 1),France,164,CF,Right,High,Med,gold,99,97,99,93,98,50,97,97.0,97.0,99.0,99.0,99.0,97.0,99.0,97.0,99.0,84.0,83.0,98.0,86.0,93.0,91.0,90.0,99.0,99.0,99.0,99.0,55.0,99.0,33.0,25.0,56.0,99.0,99.0,99.0,90.0,NaN,NaN,NaN,NaN,NaN,97
2,18982,54249,Zinedine Zidane,185,77,51,FUT ICONS,Icons,France,171,CAM,Right,Med,Med,gold,99,92,96,99,97,87,90,93.0,92.0,96.0,95.0,95.0,99.0,97.0,94.0,99.0,99.0,99.0,99.0,99.0,99.0,88.0,90.0,99.0,99.0,99.0,99.0,95.0,99.0,88.0,72.0,83.0,87.0,94.0,92.0,83.0,NaN,NaN,NaN,NaN,NaN,92
3,18730,54005,Pelé,173,70,82,FUT ICONS,Icons,Brazil,153,LW,Right,High,Med,gold,99,96,97,94,99,61,78,96.0,96.0,98.0,99.0,95.0,95.0,96.0,94.0,98.0,91.0,90.0,97.0,89.0,90.0,97.0,96.0,99.0,99.0,99.0,99.0,68.0,96.0,54.0,50.0,56.0,90.0,91.0,78.0,61.0,NaN,NaN,NaN,NaN,NaN,96
4,19001,54277,Robert Lewandowski,185,81,35,FC Barcelona,LaLiga Santander,Poland,164,ST,Right,High,Med,gold,99,97,99,92,97,53,99,98.0,97.0,99.0,99.0,99.0,99.0,99.0,99.0,94.0,83.0,99.0,98.0,82.0,92.0,87.0,92.0,99.0,99.0,97.0,99.0,60.0,99.0,52.0,24.0,42.0,99.0,97.0,99.0,99.0,NaN,NaN,NaN,NaN,NaN,97
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19030,14789,47568,Jiahui Zhang,182,0,20,Hebei CFFC,Chinese FA Super L. (CHN 1),China PR,0,CM,Right,Med,Med,bronze,46,54,37,46,46,43,51,48.0,58.0,52.0,30.0,51.0,33.0,36.0,42.0,44.0,38.0,35.0,53.0,50.0,33.0,61.0,62.0,41.0,45.0,42.0,43.0,40.0,49.0,45.0,46.0,40.0,51.0,43.0,54.0,51.0,NaN,NaN,NaN,NaN,NaN,53
19031,14790,47577,Ziye Zhao,180,0,19,Hebei CFFC,Chinese FA Super L. (CHN 1),China PR,0,RW,Right,Med,Med,bronze,46,63,48,40,47,24,44,67.0,60.0,40.0,49.0,64.0,31.0,43.0,54.0,42.0,48.0,35.0,38.0,33.0,39.0,54.0,66.0,39.0,45.0,47.0,40.0,21.0,36.0,21.0,25.0,23.0,41.0,42.0,52.0,26.0,NaN,NaN,NaN,NaN,NaN,63
19032,15077,47296,Antonio D'Silva,182,0,23,Odisha FC,Indian Super League (IND 1),India,0,GK,Right,Med,Med,bronze,46,19,13,17,28,11,32,17.0,21.0,6.0,8.0,35.0,10.0,10.0,14.0,23.0,14.0,13.0,21.0,21.0,14.0,36.0,48.0,35.0,14.0,8.0,38.0,13.0,11.0,13.0,12.0,7.0,53.0,18.0,33.0,26.0,51.0,45.0,46.0,41.0,50.0,19
19033,13807,47579,Junjie Wu,188,0,20,Guangzhou R&F,Chinese FA Super L. (CHN 1),China PR,0,LB,Left,Med,Med,bronze,46,55,25,29,34,48,57,54.0,56.0,31.0,19.0,33.0,25.0,25.0,31.0,31.0,30.0,26.0,30.0,25.0,24.0,45.0,48.0,42.0,32.0,30.0,35.0,46.0,42.0,51.0,45.0,50.0,56.0,53.0,64.0,46.0,NaN,NaN,NaN,NaN,NaN,55


In [31]:
# List of numerical columns
numerical_cols = [
    'height', 'weight', 'age', 'overallRating', 'pace', 'shooting',
    'passing', 'dribbling', 'defending', 'physicality',
    'pace_acceleration', 'pace_sprintSpeed', 'shooting_positioning',
    'shooting_finishing', 'shooting_shotPower', 'shooting_longShots',
    'shooting_volleys', 'shooting_penalties', 'passing_vision',
    'passing_crossing', 'passing_freeKickAccuracy', 'passing_shortPassing',
    'passing_longPassing', 'passing_curve', 'dribbling_agility',
    'dribbling_balance', 'dribbling_reactions', 'dribbling_ballControl',
    'dribbling_dribbling', 'dribbling_composure', 'defending_interceptions',
    'defending_headingAccuracy', 'defending_standingTackle',
    'defending_slidingTackle', 'defending_defenseAwareness',
    'phsyicality_jumping', 'physicality_stamina', 'physicality_strength',
    'physicality_aggression', 'goalkeeper_diving', 'goalkeeper_handling',
    'goalkeeper_kicking', 'goalkeeper_positioning', 'goalkeeper_reflexes',
    'goalkeeper_speed'
]

# List of categorical columns
categorical_cols = [
     'position', 'foot',
    'attackWorkRate', 'defenseWorkRate', 'cardColor'
]
player_ratings_scaled = player_ratings.copy()

# Handle missing values for numerical columns with mean of that column
for col in numerical_cols:
    player_ratings_scaled[col].fillna(player_ratings_scaled[col].mean(), inplace=True)

# Handle missing values for categorical columns with mode (most frequent value) of that column
for col in categorical_cols:
    player_ratings_scaled[col].fillna(player_ratings_scaled[col].mode()[0], inplace=True)

# Standardize numerical columns
scaler = StandardScaler()
player_ratings_scaled[numerical_cols] = scaler.fit_transform(player_ratings_scaled[numerical_cols])

# One-hot encode categorical columns
player_ratings_scaled = pd.get_dummies(player_ratings_scaled, columns=categorical_cols)
player_ratings_scaled

,id,futbin_id,name,height,weight,age,club,league,nation,rarity,overallRating,pace,shooting,passing,dribbling,defending,physicality,pace_acceleration,pace_sprintSpeed,shooting_positioning,shooting_finishing,shooting_shotPower,shooting_longShots,shooting_volleys,shooting_penalties,passing_vision,passing_crossing,passing_freeKickAccuracy,passing_shortPassing,passing_longPassing,passing_curve,dribbling_agility,dribbling_balance,dribbling_reactions,dribbling_ballControl,dribbling_dribbling,dribbling_composure,defending_interceptions,defending_headingAccuracy,defending_standingTackle,defending_slidingTackle,defending_defenseAwareness,phsyicality_jumping,physicality_stamina,physicality_strength,physicality_aggression,goalkeeper_diving,goalkeeper_handling,goalkeeper_kicking,goalkeeper_positioning,goalkeeper_reflexes,goalkeeper_speed,position_CAM,position_CB,position_CDM,position_CF,position_CM,position_GK,position_LB,position_LM,position_LW,position_LWB,position_RB,position_RM,position_RW,position_RWB,position_ST,foot_Left,foot_Right,attackWorkRate_High,attackWorkRate_Low,attackWorkRate_Med,defenseWorkRate_High,defenseWorkRate_Low,defenseWorkRate_Med,cardColor_bronze,cardColor_gold,cardColor_silver
0,18949,54231,Kylian Mbappé,0.069935,1.521727,-0.441774,73,16,18,16,3.006927,1.798925,2.238685,1.951703,2.170112,-0.236559,1.480001,1.938032,1.951400,2.042407,2.227204,2.423348,1.996066,2.370765,2.334378,2.294032,1.859971,1.727380,2.119865,1.499414,1.969703,2.044723,1.575250,2.819737,2.013257,1.950297,2.488115,-0.091204,1.683354,-0.379519,-0.407687,-0.807338,1.558578,1.887697,1.453339,0.774599,4.773320e-15,0.000000,-9.959911e-15,-4.450662e-15,-4.576791e-15,1.972583,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,1,0,0,0,1,0,0,1,0
1,18981,54251,Karim Benzema,0.346944,1.756129,0.756160,607,350,18,164,3.006927,1.686473,2.287935,2.007656,2.110459,0.001856,2.141186,1.812729,1.824902,2.042407,2.227204,2.423348,2.137732,2.569602,2.561278,2.545640,1.550041,1.884993,2.183031,1.746574,2.019676,1.534374,1.509774,2.819737,2.013257,1.950297,2.488115,0.235329,2.330767,-0.838881,-1.109638,0.294395,2.385417,1.887697,2.362018,1.739693,4.773320e-15,0.000000,-9.959911e-15,-4.450662e-15,-4.576791e-15,1.847251,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,1,0,0,0,0,1,0,1,0
2,18982,54249,Zinedine Zidane,0.346944,1.638928,2.498610,112658,2118,18,171,3.006927,1.405342,2.140185,2.343375,2.050807,1.766128,1.678356,1.562121,1.508655,1.901442,2.041350,2.161580,2.232176,2.470184,2.391103,2.545640,2.324865,2.725591,2.246198,2.549841,2.319514,1.342993,1.509774,2.819737,2.013257,1.950297,2.488115,2.101233,2.330767,1.687609,1.089808,1.587735,1.483411,1.592045,1.831955,1.342301,4.773320e-15,0.000000,-9.959911e-15,-4.450662e-15,-4.576791e-15,1.533922,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0,0,1,0,1,0
3,18730,54005,Pelé,-0.761091,1.433825,5.874606,112658,2118,54,153,3.006927,1.630247,2.189435,2.063609,2.170112,0.526369,0.884935,1.750077,1.761652,1.995418,2.227204,2.161580,2.043288,2.420474,2.391103,2.482738,1.911626,2.252754,2.119865,1.931943,1.869758,1.917136,1.902631,2.819737,2.013257,1.950297,2.488115,0.841748,2.168914,0.125779,0.060280,0.294395,1.708912,1.414654,0.771829,0.093357,4.773320e-15,0.000000,-9.959911e-15,-4.450662e-15,-4.576791e-15,1.784585,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,1,0,0,0,0,1,0,1,0
4,19001,54277,Robert Lewandowski,0.346944,1.756129,0.756160,241,53,37,164,3.006927,1.686473,2.287935,1.951703,2.050807,0.144905,2.273422,1.875380,1.824902,2.042407,2.227204,2.423348,2.232176,2.569602,2.674728,2.231130,1.498386,2.725591,2.183031,1.499414,1.969703,1.279199,1.640726,2.819737,2.013257,1.848830,2.488115,0.468567,2.330767,0.033907,-1.156435,-0.376225,2.385417,1.769436,2.362018,2.250625,4.773320e-15,0.000000,-9.959911e-15,-4.450662e-15,-4.576791e-15,1.847251,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,1,0,0,0,0,1,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,..

In [32]:
# Only using the names that are present in both dfs
combined_df = pd.merge(player_valuations_adjusted_scaled, player_ratings_scaled, on='name', how='inner')
combined_df

,name,last_season,adjusted_value,id,futbin_id,height,weight,age,club,league,nation,rarity,overallRating,pace,shooting,passing,dribbling,defending,physicality,pace_acceleration,pace_sprintSpeed,shooting_positioning,shooting_finishing,shooting_shotPower,shooting_longShots,shooting_volleys,shooting_penalties,passing_vision,passing_crossing,passing_freeKickAccuracy,passing_shortPassing,passing_longPassing,passing_curve,dribbling_agility,dribbling_balance,dribbling_reactions,dribbling_ballControl,dribbling_dribbling,dribbling_composure,defending_interceptions,defending_headingAccuracy,defending_standingTackle,defending_slidingTackle,defending_defenseAwareness,phsyicality_jumping,physicality_stamina,physicality_strength,physicality_aggression,goalkeeper_diving,goalkeeper_handling,goalkeeper_kicking,goalkeeper_positioning,goalkeeper_reflexes,goalkeeper_speed,position_CAM,position_CB,position_CDM,position_CF,position_CM,position_GK,position_LB,position_LM,position_LW,position_LWB,position_RB,position_RM,position_RW,position_RWB,position_ST,foot_Left,foot_Right,attackWorkRate_High,attackWorkRate_Low,attackWorkRate_Med,defenseWorkRate_High,defenseWorkRate_Low,defenseWorkRate_Med,cardColor_bronze,cardColor_gold,cardColor_silver
0,Aaron Boupendza,0.339628,0.737044,12620,45349,-0.114737,-0.617199,-0.115065,111674,350,115,0,0.677278,0.955532,1.204436,0.552874,0.977059,-1.094853,0.620461,1.060906,1.002661,1.008668,1.205006,1.245390,1.051623,1.326868,1.256604,0.469872,0.516941,1.359619,0.351202,0.263618,1.170137,1.279199,0.527630,0.383738,0.778061,1.037096,0.584882,-1.397337,0.658284,-1.252307,-1.343622,-1.334254,1.107575,0.823350,0.468936,0.320437,4.773320e-15,0.000000,-9.959911e-15,-4.450662e-15,-4.576791e-15,1.032595,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,1,0,0,0,0,1,0,1,0
1,Aaron Chapman,0.339628,-0.354754,3620,30529,2.008995,-0.617199,0.538354,361,61,14,1,-0.740770,-2.024456,-1.750560,-1.965019,-1.826615,-1.714732,-1.230856,-2.322296,-2.286299,-2.139537,-1.861586,-1.306851,-2.017815,-1.953952,-1.636369,-1.228483,-2.117463,-1.740087,-2.428126,-1.713657,-1.978158,-2.357039,-0.978325,-1.128261,-2.590657,-2.463506,-1.114434,-1.723870,-2.309023,-1.803541,-1.671199,-1.382156,-0.696438,-1.719255,-0.515466,-1.780060,3.195954e-01,-2.024294,-2.717737e+00,-1.997010e+00,-6.590943e-01,-2.288694,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0,0,1,1,0,0
2,Aaron Connolly,0.885587,0.884041,2275,33254,-0.576418,-0.617199,-0.550677,205,32,25,0,-0.031746,0.449496,0.810436,-0.398330,0.499838,-1.428634,-0.040724,0.622343,0.306920,0.679751,0.786835,0.721854,0.721069,0.978902,0.519180,-0.725267,-0.722778,-0.216503,0.098535,-0.972179,-1.028672,0.641263,0.593106,0.551738,0.497334,0.428296,0.041101,-1.583927,0.496431,-1.527924,-1.577605,-1.813269,0.280736,-0.004475,-0.515466,0.490748,4.773320e-15,0.000000,-9.959911e-15,-4.450662e-15,-4.576791e-15,0.468603,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0,0,1,0,1,0,0,0,1
3,Aaron Cresswell,1.158566,0.995439,461,27226,-0.576418,1.316624,0.538354,19,13,14,0,0.879856,-0.168992,0.367187,1.000499,0.738449,1.289297,0.223750,-0.004176,-0.452071,0.209869,0.136346,0.590969,0.673846,0.034424,0.405730,0.595676,1.498386,1.517231,0.856534,0.696146,1.370028,0.003326,0.593106,1.055737,0.778061,0.783430,0.720827,1.261576,0.658284,1.228247,1.417385,1.348227,1.408243,0.350308,-0.515466,0.774599,4.773320e-15,0.000000,-9.959911e-15,-4.450662e-15,-4.576791e-15,-0.220721,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,1,0,0,0,0,1,0,1,0
4,Aaron Herzog,0.066649,-0.362971,10584,34279,-0.391746,-0.617199,-0.332871,110482,2076,21,0,-0.740770,0.786854,-0.174562,0.217155,0.440185,0.001856,-0.305197,0.872950,0.749664,0.021917,-0.374753,-0.063451,-0.034485,-0.661508,0.008655,-0.033344,0.155356,-0.374115,0.288035,0.448987,-0.428997,1.024025,1.051440,-0.456262,0.385044,0.326829,-0.230790,0.002091,-0.852345,-0.057966,0.107077,0.150691,-0.019933,0.291177,-0.666913,-0.644656,4.773320e-15,0.000000,-9.959911e-15,-4.450662e-15,-4.576791e-15,0.844598,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,1,0,0,0,0